In [ ]:
from itertools import chain, cycle, islice
from collections import deque
from typing import Callable, Iterator, Tuple, List
from collections.abc import Iterable
from numbers import Number
from operator import add

# Iterator Basics

### Exercise A
1. Write a function `naturals() -> Iterator` which yields the natural numbers starting at 0. Hint: Use `yield`

In [ ]:
#@title naturals { display-mode: "form" }
def naturals(start: int = 0) -> Iterator[int]:
    """
    :param start: an integer
    :return: the naturals
    """
    while True:
        yield start
        start += 1


2. Write a function `faculty() -> Iterator` which yields the factorials starting at 1.

In [ ]:
def faculty() -> Iterator[int]:
    """
    :return: un iterator yielding the factorials
    """
    factor = 1
    current = 1
    while True:
        yield current
        current *= factor
        factor += 1

3. Write a function `take(t: Iterator, n: int) -> list` which returns a list of the first n elements of t.
Hint: Use `islice()`

In [ ]:
def take(t: Iterator, n: int) -> List[int]:
    """
    :param t: an iterator
    :param n: number of elements requested
    :return: list of first k elements with k = min(n, len(t))
    This avoids fiddling with StopIteration
    """
    return list(islice(t, n))

4. Write a function `exp_taylor() -> Iterator` which returns the Taylor series of the exponential function.
Hint: use `faculty()`

In [ ]:
def exp_taylor() -> Iterator[float]:
    """
    :return: iterator yielding the coefficients of the Taylor series of
    exp_taylor
    """
    return (1.0 / k for k in faculty())

5. Write a function `sin_taylor() -> Iterator` which returns the Taylor series of sine. Hint: use `zip()` and `cycle()`

In [ ]:
def sin_taylor() -> Iterator[float]:
    """
    :return: iterator yielding the coefficients of the Taylor series of sin
    """
    return (a * b for a, b in zip(cycle((0, 1, 0, -1)), exp_taylor()))

6. Write a function `cos_taylor() -> Iterator` which returns the Taylor series of cosine. Hint: use `zip()` and `cycle()`

In [ ]:
def cos_taylor() -> Iterator[float]:
    """
    :return: iterator yielding the coefficients of the Taylor series of sin
    """
    return (a * b for a, b in zip(cycle((1, 0, -1, 0)), exp_taylor()))

### Exercise B
1. Write a recursive version of generator `naturals()`. Hint: Use `yield from`.
What about the call stack?

In [ ]:
def naturals1(start: int = 0) -> Iterator[int]:
    """
    :param start: an integer
    :return: the naturals
    """
    yield start
    yield from naturals(start + 1)

2. Write a function `fun(f: Callable, *args: any) -> Iterator`. Here, `f` takes k arguments, e.g. k = 2, and k is the number
of arguments given. `fun` returns  `arg0, arg1, f(arg0, arg1), f(arg1, f(arg0, arg1)), ..`

In [ ]:
def fun(f: Callable, *args: any) -> Iterator[any]:
    """
    :param f: a function taking k arguments, e.g. k = 2
    :param args: k arguments accepted by f
    :return: n iterator yielding arg0, arg1, f(arg0, arg1), f(arg1, f(arg0, arg1)), ..
    """
    for arg in args:
        yield arg

    while True:
        args = args[1:] + (f(*args),)
        yield args[-1]

3. Write a function `ari(increment: Number, start: Number) -> Iterator` which returns the arithmetic series starting at `start`with
the given `increment`. Hint: Use `fun()`

In [ ]:
def ari(increment: Number , start: Number = 0) -> Iterator[Number]:
    """
    :param increment: any number
    :param start: any number, starting value
    :return: the arithmetic series staring at start
    """
    return fun(lambda x: x + increment, start)

4. Write a function `geo(factor: Number, start: Number) -> Iterator` which returns the geometric series starting at `start`with
the given `factor`. Hint: Use `fun()`

In [ ]:
def geo(factor: Number, start: Number = 1) -> Iterator[Number]:
    """
    :param factor: any number
    :param start: any number, starting value
    :return: the geometric series staring at start
    """
    return fun(lambda x: x * factor, start)

5. Write a function `fibo()` which returns the Fibonacci numbers.
Hint: Use `fun()`

In [ ]:
def fibo() -> Iterator[int]:
    """
    :return: iterator yielding the fibonacci series
    """
    return fun(add, 1, 1)

6. Write a class `Fibo()` which can be used as an Iterable and returns the Fibonacci numbers.
Hint: Overload `__iter__(self)`

In [ ]:
class Fibo(object):
    def __iter__(self):
        return fibo()

7. Write a function `alternate() -> Iterator` which alternates between the arithmetic and geometric series. It goes:
`ari0, geo0, ar1, geo1, ...`

In [ ]:
def alternate() -> Iterator[int]:
    a = ari(1)
    g = geo(2)
    while True:
        yield next(a)
        yield next(g)

8. Write a function `hamming(*ps: int) -> Iterator` which produces all multiples of `p0`, `p1`, `p2`, ...
So, hamming(2, 3, 5) yields 1, 2, 3, 4, 5, 6, 8, 9, 10, 12, ...
Hint: This solution follows Dijkstra: "An exercise attributed to R.W. Hamming":
Let `q` be the sequence of multiples produced so far.
Then append `min{p*x | x in q, p in ps, p*x > max(q)}` to `q`.
The next number to be produced is `min(q)`.

In [ ]:
def hamming(*ps: int) -> Iterator[int]:
    """
    :param ps: ps = (p0, p1, p2, ..) contains one or more integers, generally primes
    :return: an iterator yielding all multiples of p0, p1, p2, ..

    This solution follows Dijkstra: "An exercise attributed to R.W. Hamming":
    Let q be the sequence of multiples produced so far.
    Then append min{p*x | x in q, p in ps, p*x > max(q)} to q.
    The next number to be produced is min(q)
    """

    q = [1]  # q contains all numbers produced so far
    while True:
        yield q[-1]
        mq = max(q)
        q.append(min([p * x for p in ps for x in q if p * x > mq]))



### Exercise C
1. Write a function `skip_duplicates(t: Iterable) -> Iterator` which skips successive duplicates of `t`

In [ ]:
def skip_duplicates(t: Iterable) -> Iterator:
    """"
    :param t: an iterator
    :return: an iterator skipping successive duplicates
    """
    t = iter(t)
    last = next(t)
    yield last

    while True:
        x = next(t)
        if x != last:
            last = x
            yield last

2. Write a function `merge(*ts: Iterable) -> Iterator` which merges n non-descending iterators into one.
Hint: Keep a dictionary `heads` with key = given iterator t and value = last element read (i.e. head of t)

In [ ]:
def merge(*ts: Iterable) -> Iterator:
    """
    :param ts: a list of n non-descending iterables, n >= 0
    :return: merge of n iterables into one
    This is a weak merge:
    It doesn't stop before the longest iterator is exhausted
    """
    ts = [iter(t) for t in ts]
    heads = {}.fromkeys(ts)  # dictionary of last read entries, initially all None

    while True:
        # get first element of each iterable if there is one
        for t in ts:
            if heads[t] is None:
                h = take(t, 1)  # try to read next element
                heads[t] = h[0] if len(h) > 0 else None

        # get non-exhausted entries
        active_ts = [t for t in ts if heads[t] is not None]
        if not active_ts:  # no active entry left
            return
        else:
            # get non-exhausted entry with minimum value
            min_t = min(active_ts, key=lambda t: heads[t])
            yield heads[min_t]
            heads[min_t] = None  # to be updated on next loop



3. Write a recursive function `merge(s, t: Iterable) -> Iterator` which merges two non-descending iterators into one.
Hint: The solution follows exactly the list-based solution. Use `chain()`.

In [ ]:

def merge1(s, t: Iterable) -> Iterator:
    """
    :param s: a non-descending iterable,
    :param t: a non-descending iterable,
    :return: merge of s and t into one
    This is a weak merge:
    It doesn't stop before the longest iterator is exhausted
    Elegant but inefficient (max recursion depth)
    """
    s, t = iter(s), iter(t)  # get iterator from iterable
    head_s, head_t = take(s, 1), take(t, 1)  # take first element

    if not head_s:
        yield from chain(head_t, t)  # restore first element of t
    elif not head_t:
        yield from chain(head_s, s)  # restore first element of s
    elif head_s[0] <= head_t[0]:
        yield head_s[0]
        yield from merge1(s, chain(head_t, t))
    else:
        yield head_t[0]
        yield from merge1(chain(head_s, s), t)


4. Write a function `tee(t: Iterable, n: int) -> Tuple[Iterator]` which forks an iterable into n copies
to be iterated on independently. Hint: Keep a list of n deques, one for each fork. This is the buffer.
Each iterator is fed from its deque; you have to fill the deques if they are exhausted.
This is `itertools.tee`.

In [ ]:
def tee1(t: Iterable, n: int = 2) -> Tuple[Iterator]:
    """
    :param t: an iterable
    :param n: number of forks
    :return: n copies of t, to be iterated on independently
    This is a remake of itertools.tee
    """
    t = iter(t)
    buffer = [deque() for _ in range(n)]

    def gen(q: deque) -> Iterator:
        while True:
            if not q:  # when the local deque is empty
                x = next(t)  # fetch a new value and
                for d in buffer:  # load it into buffer
                    d.append(x)
            yield q.popleft()

    return tuple(gen(d) for d in buffer)



## Iterators and Iterables

All Python collections are iterables, that is, you get an iterator by

        xs = [1, 2, 3]
        t = iter(xs)
An iterator is a recipe for computing the next element. Its main operation is

       next(t)
which applies the recipe and returns the next element. This works fine as long as there are elements left;
the fourth call in the example above would raise a `StopIteration`. Each for-loop implicitly
applies `iter()` to what follows the in-clause:

        for x in xs:
            for y in xs:
                print(x, y)

Here, Python creates two iterators on `xs`, being iterated on independently; the for-loop gracefully handles
`StopIteration`.


What is important:

* There can be any number of iterators on a given collection
* Iterators are read-once; each `next` winds the iterator one notch down. There is no winding up.
Iterators can however be chained, like this:

        s = chain([x], t)
creates a new iterator with `x` in front of `t`.

* Iterators are lazy as opposed to lists which are eager. *Lazy* means, that elements are computed
not before they are need. This allows to manage infinite series.

* Iterators are hard to debug. The `take`-function can be helpful. Good idea: Write a list-based version first and
convert to iterators later.


The class `Iterator` in `Collections.abc` is an abstract base
class defining a single method `__next__()`. Iterators are ubiquitous but often invisible. They are created by
 `Iterables`, another abstract base class in `Collections.abc`.


An iterable is just an object which the iter-function accepts.
There are three ways to get one:

1. applying `iter()`to a collection
2. writing a generator
3. overloading `__iter__`

We have seen the first way, let's turn to the second.

#### Generators
A generator is a function which
contains at least one `yield`-statement, such as this one:

    def naturals() -> Iterator
        current = 0
        while True
            yield current
            current += 1

This generator would be used as follows:

    nat = naturals()
    a = next(nat)
    b = next(nat)


Here is how generators work: A call of `naturals()` produces a generator objet, `nat` in this case.
The code inside the function is executed until and excluding the first `yield`.
In the example above, `current` is set to 0, and generator is waiting for the first call of `next(nat)`.
On that call, the generator yields what it is supposed to and proceeds until and excluding the next `yield`:
`current`gets incremented, the while-loop restarts and stops. This can go on forever, but of course,
it normally doesn't. The good news is that generator gracefully handle infinity: `naturals()` delivers
an arbitrarily many natural numbers; there is to need to define an upper bound such as, say, 1e20.
It is up to the caller to call `next(nat)` as often as desired.

#### Overloading `__iter__`
# todo

#### Iterators and Iterables

Iterables produce Iterators via `ìter()` such as all Collections.
Iterator yield the next element via `next()`. By convention, all Iterators
are also Iterables; the standard implementation being to return self.
Some examples:

* `(0, 1, 2, 3)` is a tuple, no iterator
* `range(4)` is a tuple, no iterator
* `naturals()` is an iterator
* `(1 / k for k in naturals())` is an iterator


